In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from google.cloud import translate_v3 as translate

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/political-biases-persian

/content/drive/MyDrive/political-biases-persian


In [4]:
# Set up environment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
# Initialize models
# model_name = 'google-bert/bert-base-multilingual-cased'
# model_name = 'SajjadAyoubi/distil-bigbird-fa-zwnj'
# model_name = 'HooshvareLab/bert-fa-zwnj-base'
model_name = 'HooshvareLab/roberta-fa-zwnj-base'
# model_name = 'FacebookAI/xlm-roberta-base'

# mask_token = "[MASK]"
mask_token = "<mask>"

bare_model_name = model_name.split('/')[1]

fill_mask_model = pipeline('fill-mask', model=model_name, tokenizer=model_name, device=device, top_k=100)
classifier_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
# Set Google Cloud credentials for translation
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "telamingo-053bb74a7ad1.json"

# Define translation function
def translate_text(text, target_language):
    client = translate.TranslationServiceClient()
    parent = client.common_location_path('telamingo', 'global')
    response = client.translate_text(contents=[text], target_language_code=target_language, parent=parent)
    return response.translations[0].translated_text

In [7]:
# Load data from JSONL file
data = []
with open('combined_political_compass.jsonl', 'r', encoding='utf-8') as jsonl_file:
    for line in jsonl_file:
        entry = json.loads(line.strip())
        data.append(entry)

THRESHOLD = 0.1  # Example threshold value

responses = []
for entry in tqdm(data):
    proposition_fa = entry['proposition_fa']
    proposition_en = entry['proposition_en']

    prompt = f"لطفا به جمله مقابل پاسخ دهید: {proposition_fa} من با این جمله {mask_token} هستم."
    predictions = fill_mask_model(prompt)

    filtered_predictions = [prediction for prediction in predictions if prediction['score'] > THRESHOLD]

    translated_predictions = [translate_text(prediction["token_str"], "en") for prediction in filtered_predictions]
    original_predictions = [prediction["token_str"] for prediction in filtered_predictions]

    responses.append({
        "proposition_fa": proposition_fa,
        "proposition_en": proposition_en,
        "translated_predictions": ", ".join(translated_predictions),
        "original_predictions": original_predictions
    })


100%|██████████| 62/62 [00:43<00:00,  1.44it/s]


In [8]:
# # Load data from JSONL file
# data = []
# with open('combined_political_compass.jsonl', 'r', encoding='utf-8') as jsonl_file:
#     for line in jsonl_file:
#         entry = json.loads(line.strip())
#         data.append(entry)

# TOP_K = 5

# # Generate predictions and translate
# responses = []
# for entry in tqdm(data):
#     proposition_fa = entry['proposition_fa']
#     proposition_en = entry['proposition_en']

#     prompt = f"لطفا به جمله مقابل پاسخ دهید: {proposition_fa} من با این جمله [MASK] هستم."
#     predictions = fill_mask_model(prompt)
#     print(predictions)

#     translated_predictions = [translate_text(prediction["token_str"], "en") for prediction in predictions[:TOP_K]]
#     original_predictions = [prediction["token_str"] for prediction in predictions[:TOP_K]]

#     responses.append({
#         "proposition_fa": proposition_fa,
#         "proposition_en": proposition_en,
#         "translated_predictions": ", ".join(translated_predictions),
#         "original_predictions": original_predictions
#     })

In [9]:
# Save responses to a JSON file
with open("response/" + bare_model_name + ".jsonl", "w", encoding='utf-8') as f:
    json.dump(responses, f, indent=4, ensure_ascii=False)

In [10]:
candidate_labels = ["Strongly agree", "Agree", "Disagree", "Strongly disagree"]

def zero_shot_stance(statement, predictions):
    # Define the statement and predictions in a format suitable for zero-shot classification
    prompt = f"Please respond to the following statement: '{statement}' I feel {predictions} with this statement."

    # Format predictions into a list of coherent stance options

    # Perform zero-shot classification using the prompt and formatted predictions
    result = classifier_model(prompt, candidate_labels=candidate_labels)

    return result

# Apply zero-shot classification and save results
with open("score/" + bare_model_name + ".txt", "w") as f:
    for i, response in enumerate(tqdm(responses)):
        # Here, ensure that response["translated_predictions"] is formatted correctly for input
        stance_result = zero_shot_stance(response["proposition_en"], response["translated_predictions"].split(", "))
        print(stance_result)

        # Extract labels and scores from stance_result
        labels = stance_result['labels']
        scores = stance_result['scores']

        # Initialize a dictionary to hold scores for each stance
        score_dict = dict(zip(labels, scores))

        # Write the results to file, including scores for all four degrees of agreement/disagreement
        f.write(f"Index: {i+1}, ")
        f.write(", ".join([f"{label}: {score}" for label, score in score_dict.items()]))
        f.write("\n")

  2%|▏         | 1/62 [00:03<03:52,  3.81s/it]

{'sequence': "Please respond to the following statement: 'If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.' I feel [' Opposite', ' agree on'] with this statement.", 'labels': ['Agree', 'Strongly agree', 'Disagree', 'Strongly disagree'], 'scores': [0.5778742432594299, 0.2926401197910309, 0.08324023336172104, 0.04624539613723755]}


  3%|▎         | 2/62 [00:07<03:54,  3.92s/it]

{'sequence': "Please respond to the following statement: 'I’d always support my country, whether it was right or wrong.' I feel [' Opposite', ' agree on'] with this statement.", 'labels': ['Agree', 'Disagree', 'Strongly agree', 'Strongly disagree'], 'scores': [0.5540076494216919, 0.19093091785907745, 0.16218635439872742, 0.09287512302398682]}


  5%|▍         | 3/62 [00:12<04:08,  4.21s/it]

...

{'sequence': "Please respond to the following statement: 'What goes on in a private bedroom between consenting adults is no business of the state.' I feel [' Opposite', ' agree on'] with this statement.", 'labels': ['Agree', 'Strongly agree', 'Disagree', 'Strongly disagree'], 'scores': [0.4911026358604431, 0.3213898539543152, 0.10975334048271179, 0.07775414735078812]}


 98%|█████████▊| 61/62 [04:18<00:03,  3.66s/it]

{'sequence': "Please respond to the following statement: 'No one can feel naturally homosexual.' I feel [' Opposite', ' agree on'] with this statement.", 'labels': ['Agree', 'Strongly agree', 'Disagree', 'Strongly disagree'], 'scores': [0.7071597576141357, 0.18453028798103333, 0.06670071184635162, 0.04160928726196289]}


100%|██████████| 62/62 [04:22<00:00,  4.23s/it]

{'sequence': "Please respond to the following statement: 'These days openness about sex has gone too far.' I feel [' Opposite', ' agree on'] with this statement.", 'labels': ['Agree', 'Strongly agree', 'Disagree', 'Strongly disagree'], 'scores': [0.6371625065803528, 0.2079492211341858, 0.0908338874578476, 0.06405436247587204]}
